notebook to test and find best hyperparameters for various models

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV as RSCV

In [2]:
import os

pid = os.getpid()
print("PID: %i" % pid)

n_cpu = os.cpu_count()   # Number of CPUs assigned to this process
print("Number of CPUs in the system:", n_cpu)

# we won't use all the available cpu's for this script 
n_jobs = n_cpu - 2 # The number of tasks to run in parallel

# Control which CPUs are made available for this script
cpu_arg = ''.join([str(ci) + ',' for ci in list(range(n_jobs))])[:-1]
cmd = 'taskset -cp %s %i' % (cpu_arg, pid)
print("executing command '%s' ..." % cmd)
os.system(cmd)

PID: 15664
Number of CPUs in the system: 16
executing command 'taskset -cp 0,1,2,3,4,5,6,7,8,9,10,11,12,13 15664' ...


1

In [3]:
# path to cleaned data file
trainFilePath = 'data/data_train.pkl'

# Read already prepared and saved train datasets
with open(trainFilePath, 'rb') as f:
    data_train = pickle.load(f)

# Create dummy variables for categorical data
data_train = pd.get_dummies(data_train, columns=['clinical_stage', 'biopsy_gleason_gg', 'pathological_gleason_gg',
                                'pathologic_stage', 'lni', 'surgical_margin_status', 'persistent_psa',
                                'TRYSgrupes', 'PLNDO1'])

In [38]:
"""
Explodes the provided "df" dataset based on provided survival column "time" and
clips the data to be in a range [min_time; max_time] (). A new discrete survival column
will be created with name set as variable "time_discrete". "cum_event" boolean determines
if cumulative event column will be created or no.

clip(lower, upper) function will help us create a new discrete survival time column. 
If we specify lower=1 and upper=200, patients who experienced event earlier than 200th 
month will only have records till their event, on other side, if a patient survived past 
200th month, we will clip this information and will only keep information about him til 200th month.
Another example, if we specify lower=140 and upper=200, and if the person experienced event 
at 100th month, we will create records for him till 140th (lower boundary) month.
"""
def explode_data(df,max_time,time,target_column,min_time=1,
                 time_discrete='survival_time_discrete',cum_event=False):

    target_column_discrete = target_column + '_discrete'

    # We create a new time column and clip the data by provided min and max survival times
    df[time_discrete] = df[time].clip(min_time,max_time).apply(range)

    # Exploding the dataset with the created range value in new time column
    data_exploded = df.explode(time_discrete)
    data_exploded.reset_index(drop=True, inplace=True)

    # New column starts at 0, we'll increase each value by 1
    data_exploded[time_discrete] = pd.to_numeric(data_exploded[time_discrete]) + 1

    # New event column, which will indicate the last event date
    data_exploded[target_column_discrete] = (data_exploded[time_discrete] >= data_exploded[time]) * pd.to_numeric(data_exploded[target_column])
    
    if cum_event == True:
        target_column_cumulative = target_column + '_cumulative'

        # Create new event column with duplicated event values from discrete column
        data_exploded[target_column_cumulative] = data_exploded[target_column_discrete]
        
        # For cumulative events, after end_time we will have NA values, we'll replace those with event indicator
        after_survival_time = data_exploded[time_discrete] > data_exploded[time]
        data_exploded.loc[after_survival_time, target_column_discrete] = -1
        data_exploded[target_column_discrete] = data_exploded[target_column_discrete].replace(-1,np.NaN)
        data_exploded.loc[(after_survival_time & (data_exploded[target_column]==0)), target_column_cumulative] = -1
        data_exploded[target_column_cumulative] = data_exploded[target_column_cumulative].replace(-1,np.NaN)

    return data_exploded


"""
Given an exploded dataset with instant mortality probabilities "event_probability_column"
and "id_column" for grouping (optional), cumulative hazard column will be calculated
"""
def cumulative_hazard(df, event_probability_column, id_column):
    data_copy = df.copy()
    if id_column is not None:
        data_copy = data_copy[ [id_column, event_probability_column] ]
    else:
        data_copy = data_copy[ [event_probability_column] ]
    data_copy['negative_log_prob'] = np.log( 1 - data_copy[event_probability_column] )
    if id_column is not None:
        data_copy['cumulative_hazard'] = 1 - np.exp(data_copy.groupby(id_column)['negative_log_prob'].transform(pd.Series.cumsum))
    else:
        data_copy['cumulative_hazard'] = 1 - np.exp(data_copy['negative_log_prob'].cumsum())
    return data_copy['cumulative_hazard']



def add_predict_probabilities_optimized(df_exploded, target_column, model):
    """
    Given exploded datase. Adds predictend instant mortality probabilities as well as cumulative ones.
    """

    df_exploded_copy = df_exploded.copy()
    df_exploded_copy.drop(['patient_id', target_column + '_cumulative'], axis=1, inplace=True)
    
    # probabilities
    y_pred = model.predict_proba(df_exploded_copy)[:,1]
    df_exploded['mortality_instant_prob'] = y_pred

    # Cumulative hazard for each patient
    df_exploded['cumulative_hazard'] = cumulative_hazard(df_exploded,'mortality_instant_prob','patient_id')
    
    return df_exploded

## Random Forest

### RandomizedSearchCV

#### Cancer specific mortality

In [6]:
target_column = 'cancer_specific_mortality'
max_time = 200

# mts and bcr have different survival months columns
match target_column:
    case 'mts':
        time = 'survival_months_mts'
    case 'bcr':
        time = 'survival_months_bcr'
    case _:
        time = 'survival_months'

target_column_discrete = target_column + '_discrete'

# List of columns names which will be dropped from feature set before fitting the model
target_columns = ['cancer_specific_mortality', 'death_from_other_causes', 'bcr', 'mts']

# Explode the dataset
df_train_copy_exploded = explode_data(data_train.copy(), min_time=1, max_time=max_time, time=time, target_column=target_column)

# Drop targets/features from feature set
x_columns_to_drop = [target_column_discrete, 'survival_months', 'survival_months_bcr', 'survival_months_mts', 'patient_id']
x_columns_to_drop.extend(target_columns)
X_train = df_train_copy_exploded.drop(x_columns_to_drop, axis=1)    
y_train = df_train_copy_exploded[target_column_discrete]

,age,psa,biopsy_gleason,pathologic_gleason,bcr,mts,death_from_other_causes,clinical_stage_1,clinical_stage_2,clinical_stage_3,...,surgical_margin_status_0,surgical_margin_status_1,persistent_psa_0,persistent_psa_1,TRYSgrupes_0,TRYSgrupes_1,TRYSgrupes_2,PLNDO1_0,PLNDO1_1,survival_time_discrete
0,73.0,6.36,6.0,7.0,1,0,0,0,0,1,...,0,1,1,0,0,1,0,1,0,1
1,73.0,6.36,6.0,7.0,1,0,0,0,0,1,...,0,1,1,0,0,1,0,1,0,2
2,73.0,6.36,6.0,7.0,1,0,0,0,0,1,...,0,1,1,0,0,1,0,1,0,3
3,73.0,6.36,6.0,7.0,1,0,0,0,0,1,...,0,1,1,0,0,1,0,1,0,4
4,73.0,6.36,6.0,7.0,1,0,0,0,0,1,...,0,1,1,0,0,1,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250195,68.0,6.00,6.0,8.0,1,0,0,0,1,0,...,0,1,1,0,0,0,1,1,0,196
250196,68.0,6.00,6.0,8.0,1,0,0,0,1,0,...,0,1,1,0,0,0,1,1,0,197
250197,68.0,6.00,6.0,8.0,1,0,0,0,1,0,...,0,1,1,0,0,0,1,1,0,198
250198,68.0,6.00,6.0,8.0,1,0,0,0,1,0,...,0,1,1,0,0,0,1,1,0,199


In [21]:
param_grid = {'n_estimators':np.arange(150,700,50),
              'max_features':np.arange(0.1, 1, 0.1),
              'max_depth': [3, 5, 7, 9],
              'max_samples': [0.3, 0.5, 0.8],
              'min_samples_leaf': [1, 2, 3, 4, 5]}

rf_random = RSCV(RandomForestClassifier(), param_grid, n_iter=15, 
             random_state=0, verbose=2, scoring='roc_auc', n_jobs=n_jobs)
model_rf = rf_random.fit(X_train, y_train)
model_rf_best = model_rf.best_estimator_

Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [22]:
# Best parameters
model_rf.best_params_

{'n_estimators': 200,
 'min_samples_leaf': 5,
 'max_samples': 0.3,
 'max_features': 0.5,
 'max_depth': 3}

### Bayesian Optimization

#### cancer specific mortality

In [49]:
target_column = 'cancer_specific_mortality'
max_time = 216

# mts and bcr have different survival months columns
match target_column:
    case 'mts':
        time = 'survival_months_mts'
    case 'bcr':
        time = 'survival_months_bcr'
    case _:
        time = 'survival_months'

target_column_discrete = target_column + '_discrete'

# List of columns names which will be dropped from feature set before fitting the model
target_columns = ['cancer_specific_mortality', 'death_from_other_causes', 'bcr', 'mts']

# Explode the dataset
df_train_copy_exploded = explode_data(data_train.copy(), min_time=1, max_time=max_time, time=time, target_column=target_column)

# Drop targets/features from feature set
x_columns_to_drop = [target_column, target_column_discrete, 'survival_months', 'survival_months_bcr', 'survival_months_mts', 'patient_id']
x_columns_to_drop.extend(target_columns)
X_train = df_train_copy_exploded.drop(x_columns_to_drop, axis=1)    
y_train = df_train_copy_exploded[target_column_discrete]

df_train_copy_exploded_cumulative = explode_data(data_train.copy(), max_time=max_time, min_time=max_time, cum_event=True, time=time, target_column=target_column)
x_columns_to_drop_exploded_cumulative = [target_column+'_discrete', 'survival_months', 'survival_months_bcr', 'survival_months_mts']
x_columns_to_drop_exploded_cumulative.extend(target_columns)
df_train_copy_exploded_cumulative = df_train_copy_exploded_cumulative.drop(x_columns_to_drop_exploded_cumulative, axis=1)

In [50]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
from sklearn.model_selection import cross_val_score

space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.uniform('max_depth',5,20),
        'max_features': hp.choice('max_features', ['sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.uniform('n_estimators', 100, 500)
    }

def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = int(space['max_depth']),
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = int(space['n_estimators']), 
                                 )
    
    # fit the data
    model.fit(X_train, y_train)
    
    # add predicted probabilities
    df_train_copy_exploded_pred = add_predict_probabilities_optimized(df_train_copy_exploded_cumulative.copy(), target_column, model)
    
    # calculate auc
    # AUC for each cumulative slice
    # Months at which we'll check the AUC's
    months = list(range(6, max_time, 6))

    train_auc_stats = []
    for month in months:
        # --- Training data ---
        # Selecting a subset of data based on the months
        select = (df_train_copy_exploded_pred['survival_time_discrete'] == month) & pd.notna(df_train_copy_exploded_pred[target_column+'_cumulative'])
        sub_dat = df_train_copy_exploded_pred[select]

        # If in the sliced data there's a event, calculate AUC metric,
        # otherwise assign NaN value
        if sub_dat[target_column+'_cumulative'].max() == 1:
            fpr, tpr, thresholds = roc_curve(sub_dat[target_column+'_cumulative'], sub_dat['cumulative_hazard'])
            auc_stat = auc(fpr, tpr)
        else:
            auc_stat = float('NaN')
        train_auc_stats.append(auc_stat)

    auc_mean = np.nanmean(train_auc_stats)

    # We aim to maximize auc, therefore we return it as a negative value
    return {'loss': -auc_mean, 'status': STATUS_OK }

In [48]:
trials = Trials()
best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 40, #80
            trials = trials)
best

100%|██████████| 10/10 [01:09<00:00,  6.91s/trial, best loss: -0.935347568918119]


{'criterion': 0,
 'max_depth': 5.498136049182303,
 'max_features': 0,
 'min_samples_leaf': 0.007264818733396294,
 'min_samples_split': 0.05282217977945802,
 'n_estimators': 426.84675889453035}

In [29]:
best

{'criterion': 1,
 'max_depth': 200.0,
 'max_features': 3,
 'min_samples_leaf': 0.043813847274908446,
 'min_samples_split': 0.551636834827235,
 'n_estimators': 5}

#### death from other causes

In [51]:
target_column = 'death_from_other_causes'
max_time = 216

# mts and bcr have different survival months columns
match target_column:
    case 'mts':
        time = 'survival_months_mts'
    case 'bcr':
        time = 'survival_months_bcr'
    case _:
        time = 'survival_months'

target_column_discrete = target_column + '_discrete'

# List of columns names which will be dropped from feature set before fitting the model
target_columns = ['cancer_specific_mortality', 'death_from_other_causes', 'bcr', 'mts']

# Explode the dataset
df_train_copy_exploded = explode_data(data_train.copy(), min_time=1, max_time=max_time, time=time, target_column=target_column)

# Drop targets/features from feature set
x_columns_to_drop = [target_column, target_column_discrete, 'survival_months', 'survival_months_bcr', 'survival_months_mts', 'patient_id']
x_columns_to_drop.extend(target_columns)
X_train = df_train_copy_exploded.drop(x_columns_to_drop, axis=1)    
y_train = df_train_copy_exploded[target_column_discrete]

df_train_copy_exploded_cumulative = explode_data(data_train.copy(), max_time=max_time, min_time=max_time, cum_event=True, time=time, target_column=target_column)
x_columns_to_drop_exploded_cumulative = [target_column+'_discrete', 'survival_months', 'survival_months_bcr', 'survival_months_mts']
x_columns_to_drop_exploded_cumulative.extend(target_columns)
df_train_copy_exploded_cumulative = df_train_copy_exploded_cumulative.drop(x_columns_to_drop_exploded_cumulative, axis=1)

In [52]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
from sklearn.model_selection import cross_val_score

space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.uniform('max_depth',5,20),
        'max_features': hp.choice('max_features', ['sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.uniform('n_estimators', 100, 500)
    }

def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = int(space['max_depth']),
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = int(space['n_estimators']), 
                                 )
    
    # fit the data
    model.fit(X_train, y_train)
    
    # add predicted probabilities
    df_train_copy_exploded_pred = add_predict_probabilities_optimized(df_train_copy_exploded_cumulative.copy(), target_column, model)
    
    # calculate auc
    # AUC for each cumulative slice
    # Months at which we'll check the AUC's
    months = list(range(6, max_time, 6))

    train_auc_stats = []
    for month in months:
        # --- Training data ---
        # Selecting a subset of data based on the months
        select = (df_train_copy_exploded_pred['survival_time_discrete'] == month) & pd.notna(df_train_copy_exploded_pred[target_column+'_cumulative'])
        sub_dat = df_train_copy_exploded_pred[select]

        # If in the sliced data there's a event, calculate AUC metric,
        # otherwise assign NaN value
        if sub_dat[target_column+'_cumulative'].max() == 1:
            fpr, tpr, thresholds = roc_curve(sub_dat[target_column+'_cumulative'], sub_dat['cumulative_hazard'])
            auc_stat = auc(fpr, tpr)
        else:
            auc_stat = float('NaN')
        train_auc_stats.append(auc_stat)

    auc_mean = np.nanmean(train_auc_stats)

    # We aim to maximize auc, therefore we return it as a negative value
    return {'loss': -auc_mean, 'status': STATUS_OK }

In [53]:
trials = Trials()
best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 10, #80
            trials = trials)
best

100%|██████████| 10/10 [01:12<00:00,  7.24s/trial, best loss: -0.6360188999294815]


{'criterion': 1,
 'max_depth': 6.308941631995079,
 'max_features': 0,
 'min_samples_leaf': 0.25256851781175843,
 'min_samples_split': 0.10004228078151234,
 'n_estimators': 397.0599160292858}

#### mts

In [54]:
target_column = 'mts'
max_time = 216

# mts and bcr have different survival months columns
match target_column:
    case 'mts':
        time = 'survival_months_mts'
    case 'bcr':
        time = 'survival_months_bcr'
    case _:
        time = 'survival_months'

target_column_discrete = target_column + '_discrete'

# List of columns names which will be dropped from feature set before fitting the model
target_columns = ['cancer_specific_mortality', 'death_from_other_causes', 'bcr', 'mts']

# Explode the dataset
df_train_copy_exploded = explode_data(data_train.copy(), min_time=1, max_time=max_time, time=time, target_column=target_column)

# Drop targets/features from feature set
x_columns_to_drop = [target_column, target_column_discrete, 'survival_months', 'survival_months_bcr', 'survival_months_mts', 'patient_id']
x_columns_to_drop.extend(target_columns)
X_train = df_train_copy_exploded.drop(x_columns_to_drop, axis=1)    
y_train = df_train_copy_exploded[target_column_discrete]

df_train_copy_exploded_cumulative = explode_data(data_train.copy(), max_time=max_time, min_time=max_time, cum_event=True, time=time, target_column=target_column)
x_columns_to_drop_exploded_cumulative = [target_column+'_discrete', 'survival_months', 'survival_months_bcr', 'survival_months_mts']
x_columns_to_drop_exploded_cumulative.extend(target_columns)
df_train_copy_exploded_cumulative = df_train_copy_exploded_cumulative.drop(x_columns_to_drop_exploded_cumulative, axis=1)

In [55]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
from sklearn.model_selection import cross_val_score

space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.uniform('max_depth',5,20),
        'max_features': hp.choice('max_features', ['sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.uniform('n_estimators', 100, 500)
    }

def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = int(space['max_depth']),
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = int(space['n_estimators']), 
                                 )
    
    # fit the data
    model.fit(X_train, y_train)
    
    # add predicted probabilities
    df_train_copy_exploded_pred = add_predict_probabilities_optimized(df_train_copy_exploded_cumulative.copy(), target_column, model)
    
    # calculate auc
    # AUC for each cumulative slice
    # Months at which we'll check the AUC's
    months = list(range(6, max_time, 6))

    train_auc_stats = []
    for month in months:
        # --- Training data ---
        # Selecting a subset of data based on the months
        select = (df_train_copy_exploded_pred['survival_time_discrete'] == month) & pd.notna(df_train_copy_exploded_pred[target_column+'_cumulative'])
        sub_dat = df_train_copy_exploded_pred[select]

        # If in the sliced data there's a event, calculate AUC metric,
        # otherwise assign NaN value
        if sub_dat[target_column+'_cumulative'].max() == 1:
            fpr, tpr, thresholds = roc_curve(sub_dat[target_column+'_cumulative'], sub_dat['cumulative_hazard'])
            auc_stat = auc(fpr, tpr)
        else:
            auc_stat = float('NaN')
        train_auc_stats.append(auc_stat)

    auc_mean = np.nanmean(train_auc_stats)

    # We aim to maximize auc, therefore we return it as a negative value
    return {'loss': -auc_mean, 'status': STATUS_OK }

In [56]:
trials = Trials()
best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 10, #80
            trials = trials)
best

100%|██████████| 10/10 [01:15<00:00,  7.58s/trial, best loss: -0.890629556166743]


{'criterion': 1,
 'max_depth': 16.810243822100986,
 'max_features': 0,
 'min_samples_leaf': 0.03125048473423414,
 'min_samples_split': 0.22516444376211764,
 'n_estimators': 353.6896846350636}

#### bcr

In [57]:
target_column = 'bcr'
max_time = 216

# mts and bcr have different survival months columns
match target_column:
    case 'mts':
        time = 'survival_months_mts'
    case 'bcr':
        time = 'survival_months_bcr'
    case _:
        time = 'survival_months'

target_column_discrete = target_column + '_discrete'

# List of columns names which will be dropped from feature set before fitting the model
target_columns = ['cancer_specific_mortality', 'death_from_other_causes', 'bcr', 'mts']

# Explode the dataset
df_train_copy_exploded = explode_data(data_train.copy(), min_time=1, max_time=max_time, time=time, target_column=target_column)

# Drop targets/features from feature set
x_columns_to_drop = [target_column, target_column_discrete, 'survival_months', 'survival_months_bcr', 'survival_months_mts', 'patient_id']
x_columns_to_drop.extend(target_columns)
X_train = df_train_copy_exploded.drop(x_columns_to_drop, axis=1)    
y_train = df_train_copy_exploded[target_column_discrete]

df_train_copy_exploded_cumulative = explode_data(data_train.copy(), max_time=max_time, min_time=max_time, cum_event=True, time=time, target_column=target_column)
x_columns_to_drop_exploded_cumulative = [target_column+'_discrete', 'survival_months', 'survival_months_bcr', 'survival_months_mts']
x_columns_to_drop_exploded_cumulative.extend(target_columns)
df_train_copy_exploded_cumulative = df_train_copy_exploded_cumulative.drop(x_columns_to_drop_exploded_cumulative, axis=1)

In [58]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
from sklearn.model_selection import cross_val_score

space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.uniform('max_depth',5,20),
        'max_features': hp.choice('max_features', ['sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.uniform('n_estimators', 100, 500)
    }

def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = int(space['max_depth']),
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = int(space['n_estimators']), 
                                 )
    
    # fit the data
    model.fit(X_train, y_train)
    
    # add predicted probabilities
    df_train_copy_exploded_pred = add_predict_probabilities_optimized(df_train_copy_exploded_cumulative.copy(), target_column, model)
    
    # calculate auc
    # AUC for each cumulative slice
    # Months at which we'll check the AUC's
    months = list(range(6, max_time, 6))

    train_auc_stats = []
    for month in months:
        # --- Training data ---
        # Selecting a subset of data based on the months
        select = (df_train_copy_exploded_pred['survival_time_discrete'] == month) & pd.notna(df_train_copy_exploded_pred[target_column+'_cumulative'])
        sub_dat = df_train_copy_exploded_pred[select]

        # If in the sliced data there's a event, calculate AUC metric,
        # otherwise assign NaN value
        if sub_dat[target_column+'_cumulative'].max() == 1:
            fpr, tpr, thresholds = roc_curve(sub_dat[target_column+'_cumulative'], sub_dat['cumulative_hazard'])
            auc_stat = auc(fpr, tpr)
        else:
            auc_stat = float('NaN')
        train_auc_stats.append(auc_stat)

    auc_mean = np.nanmean(train_auc_stats)

    # We aim to maximize auc, therefore we return it as a negative value
    return {'loss': -auc_mean, 'status': STATUS_OK }

In [59]:
trials = Trials()
best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 10, #80
            trials = trials)
best

100%|██████████| 10/10 [01:16<00:00,  7.69s/trial, best loss: -0.8119986054448181]


{'criterion': 0,
 'max_depth': 16.103488534169735,
 'max_features': 2,
 'min_samples_leaf': 0.05973709304919683,
 'min_samples_split': 0.09831750690258467,
 'n_estimators': 397.0952759650272}